# Elastic Net Regressor

Elastic Net Regressor é um modelo de regressão linear que utiliza o Lasso e Ridge regressors como regularizadores de features, tanto o por meio da norma L1 (Lasso) como a L2 (Ridge).

Tal combinação permite juntarmos os pontos fortes tanto do Lasso quanto do Ridge, onde no Lasso seria a parte de _feature selection_ e o Ridge o ponto forte se dá por ser um bom modelo para lidar com multicolinearidade entre variáveis - o que costuma ser comúm em problemas de regressão, onde as variáveis independentes possuem relações lineares exatas ou aproximadamente exatas com a variável alvo.

## Normas L1 e L2

Em geral, normalizadores são artifícios matemáticos utilizados para mapear valores de um vetor entre [0,∞). Em _machine learning_, esses normalizadores são responsáveis por expressar distâncias entre pontos ou vetores: este ponto está a tão perto ou tão longe deste outro ponto, de acordo com essa norma.

Na geometria, existem diversos tipos de cálculos matemáticos desenvolvidos para o cálculo de distância entre pontos num dado espaço dimensional, mas, como estamos nos referindo aos normalizadores L1 e L2, nos referimos diretamente à dois desses cálculos: Distância Euclidiana (Ridge ou L2) e a Métrica Taxicab (Lasso ou L1).

A utilização desses normalizadores dentro de um cálculo matemático de "custo" ou valor, é o que chamamos de regularização.

## Processamento dos dados

In [1]:
import pandas as pd

# As colunas estão separadas pelo símbolo ';' #. 
weather_history = pd.read_csv('rdu-weather-history.csv', sep=';')


# Substituindo as string por números binários -- No = 0.0 e Yes = 1.0
weather_history = weather_history.replace('No', 0.0)
weather_history = weather_history.replace('Yes', 1.0)

from sklearn.preprocessing import Imputer
import numpy as np

null_values = ['temperaturemin', 'temperaturemax', 'precipitation', 'snowfall', 'snowdepth', 'avgwindspeed',
             'fastest2minwinddir', 'fastest2minwindspeed', 'fastest5secwinddir', 'fastest5secwindspeed'
            ]
#Substituindo os valores nulos pela média dos valores referentes a feature
imputer = Imputer(missing_values=np.nan, strategy='median', axis=0)
for i in null_values:
    weather_history[[i]]= imputer.fit_transform(weather_history[[i]])

## Separação por _season_

In [2]:
weather_history['date'] = weather_history['date'].replace({'-':''}, regex=True)
season_array = np.zeros(4137)
meantemp_array = np.zeros(4137)

for i in range(0, 4137):
    #formato: MMDD
    monthDay = int(weather_history['date'][i][4:8])
    
    if monthDay >= 1201 or monthDay < 301:
        season = 0 #'winter'
    elif monthDay >= 301 and monthDay < 601:
        season = 1 #'spring'
    elif monthDay >= 601 and monthDay < 901:
        season = 2 #'summer'
    elif monthDay >= 901 and monthDay < 1201:
        season = 3 #'autumn'    
    
    season_array[i] = season

weather_history['season'] = season_array
winter_weather_history = weather_history[weather_history['season'] == 0]
summer_weather_history = weather_history[weather_history['season'] == 1]
spring_weather_history = weather_history[weather_history['season'] == 2]
autumn_weather_history = weather_history[weather_history['season'] == 3]

In [3]:
from sklearn.model_selection import train_test_split
def compute(regressor, dataset, hasSeason = False):
    X = dataset.drop('temperaturemin', axis=1)
    X = X.drop('date', axis=1)
    if 'season' in dataset:
        X = X.drop('season', axis=1)
    y = dataset['temperaturemin'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    regressor.fit(X_train, y_train)
    print("\nTraining set score: {:.3f}".format(regressor.score(X_train, y_train)))
    print("Test set score: {:.3f}".format(regressor.score(X_test, y_test)))
    print("Number of used features: {}\n".format(np.sum(regressor.coef_ != 0)))

### Utilizando o Modelo Elastic Net

In [13]:
from sklearn.linear_model import ElasticNet
# for i in np.linspace(0, 1, 11):
regr = ElasticNet(random_state=0, alpha=0.2, l1_ratio=0.5)
print("Winter separadamente:")
compute(regr, winter_weather_history)
print("Summer separadamente:")
compute(regr, summer_weather_history)
print("Spring separadamente:")
compute(regr, spring_weather_history)
print("Autumn separadamente:")
compute(regr, autumn_weather_history)
print("Dataset em conjunto:")
compute(regr, weather_history)

Winter separadamente:

Training set score: 0.714
Test set score: 0.680
Number of used features: 13

Summer separadamente:

Training set score: 0.725
Test set score: 0.698
Number of used features: 12

Spring separadamente:

Training set score: 0.419
Test set score: 0.299
Number of used features: 11

Autumn separadamente:

Training set score: 0.790
Test set score: 0.788
Number of used features: 14

Dataset em conjunto:

Training set score: 0.863
Test set score: 0.865
Number of used features: 12



# Métricas do Modelo

In [5]:
from sklearn.metrics import mean_absolute_error

X = weather_history.drop('temperaturemin', axis=1)
X = X.drop('date', axis=1)
y = weather_history['temperaturemin'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

regr.fit(X_train, y_train)

test_prediction = regr.predict(X_test)
train_prediction = regr.predict(X_train)

print("MAE: {:.3f}".format(mean_absolute_error(y_test, test_prediction)))
from sklearn.metrics import mean_squared_error
train_RMSE = np.sqrt(mean_squared_error(y_train, train_prediction))
test_RMSE = np.sqrt(mean_squared_error(y_test, test_prediction))
print('Training RMSE: {:.3f}'.format(train_RMSE))
print('Test RMSE: {:.3f}'.format(test_RMSE))

MAE: 4.932
Training RMSE: 6.175
Test RMSE: 6.101


# Referências

https://en.wikipedia.org/wiki/Taxicab_geometry

http://www.sthda.com/english/articles/37-model-selection-essentials-in-r/153-penalized-regression-essentials-ridge-lasso-elastic-net/

https://www.kaggle.com/residentmario/l1-norms-versus-l2-norms

http://www.chioka.in/differences-between-l1-and-l2-as-loss-function-and-regularization/